19. Para cada categoría, indicar cuál es la aplicación que tiene mayor cantidad de reviews con sentimiento negativo (⭐⭐)

In [ ]:
!pip install pyspark
!apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
# Atencion, utilizo estos metodos para separar bien la forma en la que se lee del csv, ya que sino se lee de manera incorrecta
app_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore.csv', header=True, inferSchema=True)
app_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore_user_reviews.csv', header=True, inferSchema=True)
rdd_app_data = app_data.rdd
rdd_app_reviews = app_reviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
negative_reviews_cached = rdd_app_reviews.filter(lambda row: row["Sentiment"].lower() != 'nan' and row["Sentiment"] == "Negative").distinct().cache()
# Hago esto para poder "droppear" las repetidas. Use este metodo porque distinct elimina la fila si y solo si todos los campos de dos filas coinciden, cosa que
# no pasa en este caso. Hay apps que tienen todos los campos iguales y difieren por algunas reviews, lo que puede afectar al resultado final en este caso.
def get_key(x):
    return "{0}".format(x["App"])
rdd_app_data_cached = rdd_app_data.map(lambda x: (get_key(x), x)).reduceByKey(lambda x,y: x).cache()

Voy a contar por c/ app la cantidad de veces que aparece (ya que son las que tienen sentimientos negativos en base al filtro aplicado anteriormente), le voy a hacer un join (por supuesto que para eso lo dejo en formato clave-valor), post-join acomodo para que quede la categoria como clave y ejecuto un reduceByKey para calcular el maximo

In [ ]:
negative_reviews_cached_count = negative_reviews_cached.map(lambda row: (row["App"], 1)).reduceByKey(lambda x, y: x + y)

# Obtengo un rdd de app_data en formato clave-valor  para hacer el join, donde la clave es App (para poder hacer el join)
rdd_app_category = rdd_app_data_cached.map(lambda row: (row[1]["App"], row[1]["Category"]))

rdd_merged = negative_reviews_cached_count.join(rdd_app_category) # post-join queda asi: ("App", (cant_negativos, "Category"))
 # Hago que la key sea la categoria. Posteriormente obtengo el maximo de cada uno y hago collect porque se trata de un conjunto acotado
result = rdd_merged.map(lambda row: (row[1][1], (row[0],row[1][0])))\
.reduceByKey(lambda x, y: x if x[1] > y[1] else y).collect()
result

[('NEWS_AND_MAGAZINES', ('CNN Breaking US & World News', 15)),
 ('MEDICAL', ('Delta Dental', 20)),
 ('SOCIAL', ('Facebook', 59)),
 ('TOOLS', ('Account Manager', 18)),
 ('ENTERTAINMENT', ('Colorfy: Coloring Book for Adults - Free', 33)),
 ('FINANCE', ('Citizens Bank Mobile Banking', 19)),
 ('COMMUNICATION', ('Azar', 25)),
 ('EVENTS', ('DroidAdmin for Android - Advice', 12)),
 ('FAMILY', ('Bad Piggies', 37)),
 ('LIBRARIES_AND_DEMO', ('Aviary Stickers: Free Pack', 12)),
 ('PARENTING', ('Baby Name Together', 6)),
 ('EDUCATION', ('C++ Programming', 10)),
 ('PHOTOGRAPHY', ('Google Photos', 24)),
 ('LIFESTYLE', ('H&M', 16)),
 ('MAPS_AND_NAVIGATION', ('FindShip', 9)),
 ('BEAUTY', ('BestCam Selfie-selfie, beauty camera, photo editor', 27)),
 ('COMICS', ('DC Comics', 1)),
 ('HEALTH_AND_FITNESS', ('Eat Fit - Diet and Health Free', 21)),
 ('PRODUCTIVITY', ('Adobe Acrobat Reader', 20)),
 ('TRAVEL_AND_LOCAL',
  ('Expedia Hotels, Flights & Car Rental Travel Deals', 16)),
 ('SHOPPING', ('Gyft - Mobile